In [1]:
import time
import threading

In [2]:
class TimeoutError(Exception):
    pass

class TimeoutMonitor:
    def __init__(self, timeout=0, target=None, args=(), monitor_key="monitor", debug=False):
        self.is_timeout = False
        self.target = target
        self.args = args
        self.timeout = timeout
        self.target_respond = None
        self.monitor_key = monitor_key
        self.debug = debug
    
    def __call__(self, target=None):
        self.target = target or self.target
        if target:  # using as decorator 
            def wrapper(*args, **kwargs):
                kwargs[self.monitor_key] = self
                threading.Thread(target=self.counter, args=(self.timeout, False)).start()
                respond = target(*args, **kwargs)
                self.target_respond = respond
            return wrapper
        else:  # using as function
            threading.Thread(target=self.counter, args=(self.timeout, True)).start()
            respond = self.target(*self.args)
            self.target_respond = respond
            
    def counter(self, end=5, raise_e=False):
        if not end: return
        
        start = time.time()
        count = 0
        p_count = 1
        
        while (count < end) and not self.target_respond:
            if self.debug and (count > p_count):
                print('[ debug ] counter: %s %s' % (p_count, count))
                p_count += 1
            count = time.time() - start
        if self.debug:
            print('[ debug ] counter: %s %s' % (p_count, count))
        if not self.target_respond:
            self.is_timeout = True
            if raise_e:
                raise TimeoutError()

In [3]:
# # using as function
# def meow():
#     looping = 0
#     limit = 10

#     while looping < limit:
# #         if self.is_timeout:
# #             raise TimeoutError
# #         else:
#         print('meow ...')
#         time.sleep(1)
# try:            
#     TimeoutMonitor(3, meow)()
# except TimeoutError:
#     print('timeout !')

In [4]:
@TimeoutMonitor(6, debug=True)
def time_costing_work(monitor=None):  # monitor is necessary, but can be passed in with anything
    # while running here, monitor is TimeoutMonitor instance itself
    # which means monitor object has an attribute `is_timeout`
    # which can be used for checking whether timeout or not

    looping = 0
    limit = 10  # this example work will cost 10 secs while we set a 6 secs timeout

    while looping < limit:
        if monitor.is_timeout:  # use monitor to check whether timeout or not
            raise TimeoutError
        else:
            print('working ...')
            time.sleep(1)
        looping += 1
    
    print('normal !')
    return True

try:
    time_costing_work()
except TimeoutError:
    print('timeout !')

working ...
[ debug ] counter: 1 1.000000238418579
working ...
[ debug ] counter: 2 2.000000238418579
working ...
[ debug ] counter: 3 3.000000238418579
working ...
[ debug ] counter: 4 4.000000238418579
working ...
[ debug ] counter: 5 5.000000238418579
working ...
[ debug ] counter: 6 6.000000238418579
timeout !
